In [4]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pydicom
import imageio
import os
from glob import glob
import nibabel as nib
from tqdm import tqdm
import json
import nibabel as nib
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import nibabel as nib
from glob import glob
from tqdm import tqdm
import pandas as pd
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [13]:

data_dir='../../data/250513 C CT/'
save_dir='../../data/ct_nii/'
df=pd.read_csv('../../data/filter_ct_df.csv')

def window_and_normalize(img, dcm):
    # HU 변환
    slope = dcm.RescaleSlope if 'RescaleSlope' in dcm else 1
    intercept = dcm.RescaleIntercept if 'RescaleIntercept' in dcm else 0
    img = img * slope + intercept

    # WL/WW 적용
    wl = dcm.WindowCenter[0] if isinstance(dcm.WindowCenter, pydicom.multival.MultiValue) else dcm.WindowCenter
    ww = dcm.WindowWidth[0] if isinstance(dcm.WindowWidth, pydicom.multival.MultiValue) else dcm.WindowWidth

    if wl is not None and ww is not None:
        min_val = wl - ww / 2
        max_val = wl + ww / 2
        img = np.clip(img, min_val, max_val)
        img = (img - min_val) / (max_val - min_val)
    else:
        # fallback
        img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-8)

    # [-1, 1] 스케일링
    img = img * 2 - 1
    return img

In [ ]:
folder_list=[]
for i in range(len(df)):
    folder_list.append(data_dir+df['ID'][i])
for j in tqdm(range(len(folder_list))):

    dcm_list = sorted(glob(f'{folder_list[j]}/*.dcm'))
    nii_name = os.path.basename(folder_list[j])

    volume_img = []
    for dcm_file in dcm_list:
        dcm = pydicom.dcmread(dcm_file)
        img = dcm.pixel_array
        img_norm = window_and_normalize(img, dcm)
        gray = (img_norm * 127.5 + 127.5).astype(np.uint8)
        img_proc = cv2.resize(gray, (256, 256), interpolation=cv2.INTER_LINEAR)
        volume_img.append(img_proc.astype(np.float32)/127.5-1)
        
    volume_img = np.stack(volume_img, axis=0)
    nii_img = nib.Nifti1Image(volume_img, affine=np.eye(4))
    os.makedirs(f'{save_dir}', exist_ok=True)
    nib.save(nii_img, f'{save_dir}{nii_name}.nii.gz')